In [8]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torchvision
import sys
import time

print('Pytorch version: ', torch.__version__)
print('GPU availability: ', torch.cuda.is_available())


Pytorch version:  2.0.1+cu118
GPU availability:  True


In [9]:
data_dir='./data'
BATCH_SIZE = 64

train_dataset = torchvision.datasets.MNIST(
    root=data_dir,
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True
)
test_dataset = torchvision.datasets.MNIST(
    root=data_dir,
    train=False,
    transform=torchvision.transforms.ToTensor(),
    download=True
)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False)

In [10]:
class Single_ANN(nn.Module):
    def __init__(self):
        super(Single_ANN, self).__init__()
        self.fc1 = nn.Linear(28*28,300)
        self.fc2 = nn.Linear(300,100)
        self.fc3 = nn.Linear(100, 10)
    def forward(self, x):
        x = x.view(-1, 28*28) # Flatten every image into a single vector
        x = F.relu(self.fc1(x)) #do not use activation function on last layer
        x = F.relu(self.fc2(x))
        x= self.fc3(x)
        return x

    def name(self):
        return "MLP"

In [11]:
LR = 0.001
EPOCHS = 50
model = Single_ANN().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.MSELoss()

In [12]:
best_loss = 1000000
best_path = './Models/Multilayer_MLP.pt'

for epoch in range(EPOCHS):
    # trainning
    total_loss = 0
    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        x, target = x.cuda(), target.cuda()
        target_onehot = F.one_hot(target, 10).float()
        out = model(x)
        loss = criterion(out, target_onehot)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_loss = total_loss / len(train_dataset)
    print(f'==>>> epoch: {epoch}, train loss: {avg_loss:.6f}')
    # TODO3: Based on average accuracy on validation set, save the model weights into a file
    if best_loss > avg_loss:
      best_loss = avg_loss
      torch.save(model.state_dict(), best_path)

==>>> epoch: 0, train loss: 0.000210
==>>> epoch: 1, train loss: 0.000084


In [ ]:
best_path = './Models/Multilayer_MLP.pt'
checkpoint = torch.load(f=best_path)
model.load_state_dict(checkpoint)
total_loss = 0
correct_cnt = 0
model.eval()

for batch_idx, (x, target) in enumerate(test_loader):
    x, target = x.cuda(), target.cuda()
    out = model(x)
    target_onehot = F.one_hot(target, 10).float()
    loss = criterion(out, target_onehot)

    _, pred_label = torch.max(out, 1)
    correct_cnt += (pred_label == target).sum()
    # smooth average
    total_loss += loss.item()
avg_loss = total_loss / len(test_dataset)
avg_acc = correct_cnt / len(test_dataset)

#time for raw inference
start = time.time()
for batch_idx, (x, target) in enumerate(test_loader):
    x, target = x.cuda(), target.cuda()
    out = model(x) 
end = time.time()
time = end-start
print('time: ' + str(time))
print(f'test loss: {avg_loss:.6f}, test accuracy: {avg_acc:.6f}')

time: 0.5850744247436523
test loss: 0.000620, test accuracy: 0.852800


In [ ]:
#run an infinite loop to check power (not energy)
Idle_Power = 6 #watts
while True:
    for batch_idx, (x, target) in enumerate(test_loader):
        x, target = x.cuda(), target.cuda()
        out = model(x) 

KeyboardInterrupt: 